<a href="https://colab.research.google.com/github/tysonjohn015/openFood/blob/main/P3_02_pageweb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import itertools

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import seaborn as sns

import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go

import scipy.stats as st
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import display, Markdown

import missingno as msno
from wordcloud import WordCloud

from nltk.corpus import stopwords
from IPython.display import HTML

In [ ]:
# Initialisation des options d'affichage
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

sns.set(font_scale=1.2)

pyo.init_notebook_mode()

In [ ]:
def print_md(text):
    """Affichage d'un text en Markdown"""
    display(Markdown(text))

def print_md_df_shape(df, df_origin=None):
    """
    Affichage des dimension de l'échantillon.
    Comparison avec l'échnatillon d'origine.
    """
    
    text = "Dimensions de l'échantillon :\n"
    
    if df_origin is not None:
        origin_rows_pct = int(df.shape[0] * 100 / df_origin.shape[0])
        origin_cols_pct = int(df.shape[1] * 100 / df_origin.shape[1])
        
        text += f"- **Nombre d'individus** : {df.shape[0]} ({origin_rows_pct}% du dataset complet)\n"
        text += f"- **Nombre de variables** : {df.shape[1]} ({origin_cols_pct}% du dataset complet)\n"
    else:
        text += f"- **Nombre d'individus** : {df.shape[0]}\n"
        text += f"- **Nombre de variables** : {df.shape[1]}\n"

    print_md(text)

def get_cat_var_emp_dist_df(df, c, k=None):
    """Renvoie la distribution empirique d'une variable de type catégorie"""
    if k is None:
        effectifs = list(df[c].value_counts().values)
        labels = df[c].value_counts().index.to_list()
    else:
        effectifs = list(df[c].value_counts().iloc[:k].values)
        labels = df[c].value_counts().iloc[:k].index.to_list()
        
        # Si il reste des modalités, on les aggrège ensemble
        effectif = df[c].value_counts().iloc[k:].sum()
        if effectif > 0:
            effectifs.append(effectif)
            labels.append("autres")

    tmp = pd.DataFrame({
        c: labels,
        "effectif": effectifs
    })
    
    tmp["f"] = tmp["effectif"] / df[c].count()
    tmp["F"] = tmp["f"].cumsum()
    
    return tmp

def categ_plot_pie_chart(df, c, k):
    """Affiche un pie chart de la distribution d'une variable de type catégorie"""
    tmp = get_cat_var_emp_dist_df(df, c, k)

    # On crée le pie chart
    fig = px.pie(
        tmp,
        values="effectif",
        names=c,
        title=f"Distribution de la variable {c}"
    )
    fig = go.FigureWidget(fig)
    fig.update_traces(textinfo='percent+label')
    # display(fig)
    
    return HTML(fig.to_html())

def categ_plot_bar_chart(df, c, k, log=False):
    """Affiche un bar chart de la distribution d'une variable de type catégorie"""
    tmp = get_cat_var_emp_dist_df(df, c, k)

    # On crée le bar chart
    fig = px.bar(
        tmp,
        y='effectif',
        x=c,
        text='effectif',
        title=f"Distribution de la variable {c}",
        log_y=log
    )
    fig = go.FigureWidget(fig)
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    # fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    # display(fig)
    return HTML(fig.to_html())

def num_categ_box_chart(
        df_clean,
        num_col,
        cat_col,
        title="",
        xlabel="",
        ylabel="",
        showfliers=True,
        rotation=0
    ):
    """
    Affiche un box chart d'une variable numérique en fonction des modalités
    d"une variable de type catégorie.
    """
    fig, ax = plt.subplots(figsize=(16, 16))

    meanprops = {
        'marker':'o',
        'markeredgecolor':'black',
        'markerfacecolor':'firebrick'
    }

    sns.boxplot(
        data=df_clean.sort_values(cat_col),
        x=cat_col,
        y=num_col,
        showfliers=showfliers,
        showmeans=True,
        meanprops=meanprops,
        ax=ax
    )

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    if rotation > 0:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=rotation)

    plt.show()

# Introduction

In [ ]:
# Chargement du jeu de données nettoyé
df_clean = pd.read_csv("https://raw.githubusercontent.com/tysonjohn015/openFood/main/df_clean_SMALL.csv",
    parse_dates=["last_modified_t"],
    low_memory=False
)

## P0-S0

### Présentation générale du jeu de données

Nous allons analyser un jeu de données fourni gratuitement par [Open Food Facts](https://world.openfoodfacts.org/who-we-are).
Il s'agit d'un projet collaboratif qui vise à collecter des données sur des produits alimentaires que l'on retrouve en grande surface. Le projet a été initié en France mais il est depuis devenu international.

Le projet possède une page wiki avec des nombreuses informations sur son jeu de données : [wiki](https://wiki.openfoodfacts.org/Main_Page).

Cette page web présente l'exploration et l'analyse du jeu de données nettoyé. On retrouvera 4 parties dans le menu :
- **Introduction** : ce que vous êtes actellement en train de lire.
- **Exploration des produits** : outil d'exploration visuelle des produits sous format d'un dashboard.
- **Analyse univariée** : analyse de chaque variable une à une.
- **Analyse multivariée** : analyse des liens entre les variables.

Voici un graphique résumant le jeu données avec le nombre de valeurs présentes par variable :

In [ ]:
col_val_to_label = {
    "code": "Code bar",

    "creator": "Créateur",
    "last_modified_t": "Date de dernière modification",
    "product_name": "Nom",
    "brands": "Marque",
    "countries_fr": "Pays de vente",
    "pnns_groups_1": "Catégorie",
    "pnns_groups_2": "Catégorie détaillée",

    "nutriscore_grade": "Nutri-Score",
    "nutriscore_score": "Score de nutrition",

    "ingredients_text": "Ingrédients",
    "ingredients_n": "Nombre d'ingrédients",

    "additives_fr": "Additifs",
    "additives_n": "Nombre d'additifs",
    "additifs_n_risque_faible": "Nombre d'additifs ayant un risque faible",
    "additifs_n_risque_moyen": "Nombre d'additifs ayant un risque modéré",
    "additifs_n_risque_eleve": "Nombre d'additifs ayant un risque élevé",

    "allergens": "Allergènes",
    "allergens_n": "Nombre d'allergènes",

    "energy_100g": "Energie/100g (kJ)",
    "fat_100g": "Matières grasses/100g",
    "saturated-fat_100g": "Matières grasses saturées/100g",
    "carbohydrates_100g": "Glucides/100g",
    "sugars_100g": "Sucres/100g",
    "fiber_100g": "Fibres/100g",
    "proteins_100g": "Protéines/100g",
    "salt_100g": "Sel/100g",
}

fig, ax = plt.subplots(figsize=(16, 8))

msno.bar(df_clean, ax=ax)
ax.set_title(
    """
    Représentation des valeurs présentes par variable.
    """,
    fontsize=16)
fig.axes[0].set_ylabel("Fréquence de valeurs présentes", fontsize=16)
fig.axes[1].set_ylabel("Nombre de valeurs présentes", fontsize=16)

plt.show()

Voici une brève description de chaques variables :

**Informations générales** :
- `code` : code bar du produit.
- `creator` : identifiant de celui qui a inséré le produit dans le jeu de donnée.
- `last_modified_t` : date de dernière modification.

**Informations spécifiques** :
- `product_name` : nom commercial du produit.
- `brands` : marque du produit.
- `countries_fr` : pays de vente.
- `pnns_groups_1` : catégorie.
- `pnns_groups_2` : catégorie détaillée.
- `nutriscore_grade` : Nutri-Score (A, B, C, D ou E).
- `nutriscore_score` : score de nutrition.
- `pn_beverages` : lien entre le nom du produit et la catégorie des boissons.
- `pn_cereals_and_potatoes` : lien entre le nom du produit et la catégorie des céréales et pommes de terre.
- `pn_composite_foods` : lien entre le nom du produit et la catégorie des produits composés.
- `pn_fat_and_sauces` : lien entre le nom du produit et la catégorie des sauces et matières grasses.
- `pn_fish_meat_eggs` : lien entre le nom du produit et la catégorie des poissons, des viandes et des oeufs.
- `pn_fruits_and_vegetables` : lien entre le nom du produit et la catégorie des fruits et légumes.
- `pn_milk_and_dairy_products` : lien entre le nom du produit et la catégorie des produits laitiers.
- `pn_salty_snacks` : lien entre le nom du produit et la catégorie des snacks salés.
- `pn_sugary_snacks` : lien entre le nom du produit et la catégorie des snacks sucrés.

**Ingrédients** :
- `ingredients_text` : liste des ingrédients.
- `ingredients_n` : nombre d'ingrédients.
- `additives_fr` : liste des additifs.
- `additives_n` : nombre d'additifs.
- `additifs_n_risque_faible` : nombre d'additifs ayant un risque de sur-exposition très faible ou nul.
- `additifs_n_risque_moyen` : nombre d'additifs ayant un risque modéré de sur-exposition.
- `additifs_n_risque_eleve` : nombre d'additifs ayant un risque élevé de sur-exposition.
- `allergens` : liste des allergènes.
- `allergens_n` : nombre d'allergènes.

**Informations nutritionnelles** :
- `energy_100g` : énergie aux 100g (kJ).
- `fat_100g` : quantité de matières grasses aux 100g.
- `saturated-fat_100g` : quantité de matières grasses saturées aux 100g.
- `carbohydrates_100g` : quantité de glucides aux 100g.
- `sugars_100g` : quantité de sucres aux 100g.
- `fiber_100g` : quantité de fibres aux 100g.
- `proteins_100g` : quantité de protéines aux 100g.
- `salt_100g` : quantité de sel aux 100g.

# Exploration des produits

## Section

### Filtres des produits

In [ ]:
# Création des widgets de sélection des produits
p0s1_widgets = []

options = np.sort(df_clean[df_clean["pnns_groups_1"].notna()]["pnns_groups_1"].unique()).tolist()
p0s1_pnns_groups_1_label = widgets.HTML(value="Catégorie(s) des produits :")
p0s1_pnns_groups_1 = widgets.SelectMultiple(
    options=options,
    value=options[0:1],
    rows=5
)
p0s1_widgets += [p0s1_pnns_groups_1_label, p0s1_pnns_groups_1]

options = df_clean[df_clean["countries_fr"].notna()]["countries_fr"].value_counts().index.to_list()
p0s1_countries_fr_label = widgets.HTML(value="Pays de vente :")
p0s1_countries_fr = widgets.SelectMultiple(
    options=options,
    value=options[1:2],
    rows=5
)
p0s1_widgets += [p0s1_countries_fr_label, p0s1_countries_fr]

options = np.sort(df_clean[df_clean["nutriscore_grade"].notna()]["nutriscore_grade"].unique()).tolist()
p0s1_nutriscore_grade_label = widgets.HTML(value="Nutri-Score :")
p0s1_nutriscore_grade = widgets.SelectMultiple(
    options=options,
    value=options[0:1],
    rows=5
)
p0s1_widgets += [p0s1_nutriscore_grade_label, p0s1_nutriscore_grade]

options = np.sort(df_clean[df_clean["ingredients_n"].notna()]["ingredients_n"].unique()).astype(int).tolist()
p0s1_ingredients_n_label = widgets.HTML(value="Nombre d'ingrédients :")
p0s1_ingredients_n = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(options) - 1),
)
p0s1_widgets += [p0s1_ingredients_n_label, p0s1_ingredients_n]

options = np.sort(df_clean[df_clean["additives_n"].notna()]["additives_n"].unique()).astype(int).tolist()
p0s1_additives_n_label = widgets.HTML(value="Nombre d'additifs :")
p0s1_additives_n = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(options) - 1),
)
p0s1_widgets += [p0s1_additives_n_label, p0s1_additives_n]

options = np.sort(df_clean[df_clean["allergens_n"].notna()]["allergens_n"].unique()).astype(int).tolist()
p0s1_allergens_n_label = widgets.HTML(value="Nombre d'allergène :")
p0s1_allergens_n = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(options) - 1),
)
p0s1_widgets += [p0s1_allergens_n_label, p0s1_allergens_n]

options = np.sort(df_clean[df_clean["energy_100g"].notna()]["energy_100g"].unique()).astype(int).tolist()
p0s1_energy_100g_label = widgets.HTML(value="Énergie aux 100g :")
p0s1_energy_100g = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(options) - 1),
)
p0s1_widgets += [p0s1_energy_100g_label, p0s1_energy_100g]

widgets.VBox(p0s1_widgets)

## Section

### Visualisation graphique

In [ ]:
# Création des widgets de sélection des axes du scatter plot
p0s1_fig_label = widgets.HTML(value="<u>Projection des produits sélectionnés en fonction des variables suivantes</u>")

axis_dict = {
    "Energie/100g (kJ)": "energy_100g",
    "Matières grasses/100g": "fat_100g",
    "Matières grasses saturées/100g": "saturated-fat_100g",
    "Glucides/100g": "carbohydrates_100g",
    "Sucres/100g": "sugars_100g",
    "Protéines/100g": "proteins_100g",
    "Sel/100g": "salt_100g",
    "Score de nutrition": "nutriscore_score",
}

p0s1_x_axis_label = widgets.HTML(value="Axe des abscisses :")
p0s1_x_axis = widgets.Dropdown(
    options=axis_dict,
    value="energy_100g",
)

p0s1_y_axis_label = widgets.HTML(value="Axe des ordonnées :")
p0s1_y_axis = widgets.Dropdown(
    options=axis_dict,
    value="nutriscore_score",
)

# Création du scatter plot représentant les produits
p0s1_fig = go.FigureWidget(
    [
        go.Scatter(
            y=[],
            x=[],
            mode="markers"
        )
    ]
)

p0s1_fig.data[0].marker.opacity = 0.5

margin = go.layout.Margin(l=20, r=20, b=20, t=30)
p0s1_fig = p0s1_fig.update_layout(margin=margin)

axis_hbox = widgets.HBox([
    widgets.VBox([p0s1_x_axis_label, p0s1_x_axis]),
    widgets.VBox([p0s1_y_axis_label, p0s1_y_axis]),
])

fig_vbox = widgets.VBox([p0s1_fig_label, axis_hbox, p0s1_fig])
fig_vbox.layout.align_items = "center"
fig_vbox

### Visualisation tabulaire

In [ ]:
# Création des widgets de sélection des groupes de variables du tableau
p0s1_table_label = widgets.HTML(value="<u>Comparaison des données des produits sélectionnés</u>")

p0s1_show_info = widgets.Checkbox(
    value=True,
    description='Informations spécifiques'
)

p0s1_show_ingredients = widgets.Checkbox(
    value=False,
    description='Ingrédients'
)

p0s1_show_nutriments = widgets.Checkbox(
    value=False,
    description='Informations nutritionnelles'
)

# Création du tableau représentant les produits
def p0s1_get_cols():
    cols = ["code", "product_name"]
    
    if p0s1_show_info.value:
        cols += [
            "brands",
            "countries_fr",
            "pnns_groups_1",
            "nutriscore_grade",
        ]
    
    if p0s1_show_ingredients.value:
        cols += [
            "ingredients_text",
            "ingredients_n",
            "additives_fr",
            "additives_n",
            "additifs_n_risque_faible",
            "additifs_n_risque_moyen",
            "additifs_n_risque_eleve",
            "allergens",
            "allergens_n",
        ]
    
    if p0s1_show_nutriments.value:
        cols += [
            "energy_100g",
            "fat_100g",
            "saturated-fat_100g",
            "carbohydrates_100g",
            "sugars_100g",
            "fiber_100g",
            "proteins_100g",
            "salt_100g",
        ]
    
    return {k: v for k, v in col_val_to_label.items() if k in cols}

p0s1_table = go.FigureWidget(
    [
        go.Table(
          header=dict(
                values=list(p0s1_get_cols().values()),
            ),
            cells=dict(
                values=[],
            )
        )
    ]
)

show_hbox = widgets.HBox([p0s1_show_info, p0s1_show_ingredients, p0s1_show_nutriments])
show_hbox.layout.justify_content = "center"

table_vbox = widgets.VBox([p0s1_table_label, show_hbox, p0s1_table])
table_vbox.layout.align_items = "center"
table_vbox.layout.overflow = "scroll hidden"
table_vbox

In [ ]:
# Gestion des events des widgets
def p0s1_select_products():
    mask = df_clean["pnns_groups_1"].isin(p0s1_pnns_groups_1.value)
    mask &= df_clean["countries_fr"].isin(p0s1_countries_fr.value)
    mask &= df_clean["nutriscore_grade"].isin(p0s1_nutriscore_grade.value)
    mask &= (df_clean["ingredients_n"] >= p0s1_ingredients_n.value[0]) & (df_clean["ingredients_n"] <= p0s1_ingredients_n.value[1])
    mask &= (df_clean["additives_n"] >= p0s1_additives_n.value[0]) & (df_clean["additives_n"] <= p0s1_additives_n.value[1])
    mask &= (df_clean["allergens_n"] >= p0s1_allergens_n.value[0]) & (df_clean["allergens_n"] <= p0s1_allergens_n.value[1])
    mask &= (df_clean["energy_100g"] >= p0s1_energy_100g.value[0]) & (df_clean["energy_100g"] <= p0s1_energy_100g.value[1])
    
    return df_clean[mask].reset_index()

def on_p0s1_widgets_change(change):
    tmp = p0s1_select_products()
    
    p0s1_fig_label.value = f"<u>Projection des {tmp.shape[0]} produits sélectionnés en fonction des variables suivantes</u>"
    
    with p0s1_fig.batch_update():
        p0s1_fig.data[0].x = tmp[p0s1_x_axis.value]
        p0s1_fig.data[0].y = tmp[p0s1_y_axis.value]
#         codes, uniques = pd.factorize(tmp["nutriscore_grade"])
#         p0s1_fig.data[0].marker.color = codes
#         p0s1_fig.data[0].showlegend = True
        
        p0s1_fig.update_xaxes(title_text=p0s1_x_axis.label)
        p0s1_fig.update_yaxes(title_text=p0s1_y_axis.label)
        
    p0s1_table_label.value = f"<u>Comparaison des données des {tmp.shape[0]} produits sélectionnés</u>"
        
    with p0s1_table.batch_update():
        cols = p0s1_get_cols()
        p0s1_table.data[0].header.values = list(cols.values())
        p0s1_table.data[0].cells.values = [tmp[c].values for c in list(cols.keys())]


for w in p0s1_widgets + [p0s1_x_axis, p0s1_y_axis, p0s1_show_info, p0s1_show_ingredients, p0s1_show_nutriments]:
    if w._model_name != "HTMLModel":
        w.observe(on_p0s1_widgets_change, names="value")

on_p0s1_widgets_change(None)

# Analyse univariée

## P1-S0

### Analyse des variables de type numérique

### Distribution des variables

Commençons par observer les distributions des variables de type numérique :

In [ ]:
# On calcule les indicateurs statistiques de base
df_stats = df_clean.describe().T

# On réorganise et modifie le nom des colonnes
df_stats = df_stats[[
    "mean",
    "std",
    "min",
    "max",
    "25%",
    "50%",
    "75%"
]]
df_stats.columns = [
    "moyenne",
    "écart type",
    "min",
    "max",
    "Q1",
    "Q2",
    "Q3"
]

# On ajoute le nombre de valeurs manquantes
na_nb = []
for c in df_stats.index:
    na_nb.append(df_clean[c].isna().sum())

df_stats["valeurs manquantes"] = na_nb

# On ajoute le nombre de valeurs nulles
zero_nb = []
for c in df_stats.index:
    zero_nb.append((df_clean[c] == 0.).sum())

df_stats["valeurs nulles"] = zero_nb

# On ajoute une mesure de l'asymétrie de la distribution
df_stats["skewness"] = df_clean.skew()

In [ ]:
cols = list(df_stats.index)
cols_nb = len(cols)
c_nb = 3
r_nb = np.ceil(cols_nb / c_nb).astype(int)

sz = 16 / c_nb
fig = plt.figure(figsize=(sz *  c_nb, sz * r_nb))

for i, c in enumerate(cols):
    ax = plt.subplot(r_nb, c_nb, i + 1)
    ax.set_title(f"Distribution de {c}", fontsize=14)
    
    # Si la courbe est très étalée, on utilise une échelle log
    if df_stats.loc[c, "skewness"] > 2:
        ax.set_yscale("log")
        ax.set_ylabel("Occurrence (log)", fontsize=14)
    else:
        ax.set_ylabel("Occurrence", fontsize=14)

    df_clean[c].hist(bins=20, ax=ax)
    mean_line = ax.axvline(x=df_clean[c].mean(), linewidth=3, color='g', label="moyenne", alpha=0.7)
    med_line = ax.axvline(x=df_clean[c].median(), linewidth=3, color='y', label="médiane", alpha=0.7)
    
    plt.legend(handles=[mean_line, med_line], loc='upper right')

plt.tight_layout()
plt.show()

On remarque que seule la variable `nutriscore_score` semble suivre une loi normale multimodale. Ceci semble cohérent car cette variable est correlée à `nutriscore_grade` (voir la suite de l'analyse...) qui est une variable de type catégorie possédant 5 modalités.

### Comparaison des indicateurs statistiques

- `moyenne` : moyenne arithmétique
- `écart type` : écart-type empirique
- `min` : valeur minimale
- `max` : valeur maximale
- `Q1` : 1er quartile (25% des valeurs se trouvent en dessous)
- `Q2` : 2ème quartile (50% des valeurs se trouvent en dessous)
- `Q3` : 3ème quartile (75% des valeurs se trouvent en dessous)
- `valeurs manquantes` : nombre de valeurs manquantes
- `valeurs nulles` : nombre de valeurs à 0
- `skewness` : skewness empirique indiquant l'asymétrie de la distribution

In [ ]:
# On adapte le nombre de décimals affichées en fonction de la valeur de l'indicateur statistique
display(df_stats)

Les produits ont en moyenne 14 ingrédients. Cela peut être compréhensible pour des plats cuisinés mais peut sembler beaucoup pour par exemple une mousse au chocolat.

On constate que certains produits ont jusqu'à 9 additifs considérés comme risqués pour la santé.

On constate aussi sur le tableau ci-dessus que les produits sont en moyenne principalement composés de glucides et de matières grasses.

Enfin, on remarque en observant le skewness que la distribution des valeurs est très élalée à droite sauf pour le `nutriscore_score`. Cela semble logique puisque la plupart des aliments sont composés d'une somme de nutriments. L'étalement à droite représente les produits spécifiques, comme par exemples les huiles, qui seront composés en majorité d'un seul nutriment.

### Répartition du nombre d'ingrédients par catégorie de produit

In [ ]:
num_categ_box_chart(
    df_clean,
    "ingredients_n",
    "pnns_groups_1",
    title="Répartition des valeurs de ingredients_n en fonction de pnns_groups_1",
    xlabel="Catégorie de produit",
    ylabel="Nombre d'ingrédients",
    showfliers=False,
    rotation=30
)

Sans surprise, on constate que les aliments composites (plats cuisinés etc...) sont ceux qui contiennent le plus d'ingrédients et les fuits et les légumes sont ceux qui en contiennet le moins.

### Produits ayant un nombre élevé d'additifs à risque

Voyons quelles sont les produits ayant jusqu'à 9 additifs considérés comme risqué pour la santé :

In [ ]:
cols = [
    "code",
    
    "creator",
    "last_modified_t",
    "product_name",
    "brands",
    "pnns_groups_1",

    "nutriscore_grade",

    "additives_fr",
    "additifs_n_risque_eleve",
]

df_clean[df_clean["additifs_n_risque_eleve"] == 8][cols]

Ces produits sont des snacks sucrés vendus aux US. Il est intéressant de constater qu'ils ont été mis à jour en 2020 et sont donc certainement encore vendu en magasin.

### Comparaison des distribution des nutriments

In [ ]:
cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
]

fig, ax = plt.subplots(figsize=(16, 10))

sns.boxplot(
    data=df_clean[cols],
    showfliers=False,
    showmeans=True,
    ax=ax
)

ax.set_title("Comparaison des distributions des nutriments")
ax.set_ylabel("Quantité en g")

# plt.savefig('comparaison des distributions des nutriments.png', bbox_inches='tight')
plt.show()

On peut voir visuellement ce que l'on avait constaté dans le tableau des indicateurs statistiques (il y a en moyenne plus de glucides et de matières grasses dans nos aliments).

Il est aussi intéressant de constater les valeurs des quartiles Q3 indiquant que 75% des aliments sont en dessous de ces seuil :

In [ ]:
text = ""

for i, v in df_clean[cols].quantile(0.75).items():
    text += f"- `{i}` : {v:0.2f}g\n"
    
print_md(text)

### Analyse des variables de type catégorie

### Comparaison des indicateurs statistiques

- `nombre de valeurs` : nombre de valeurs présentes
- `nombre de modalités` : nombre de modalités de la variable
- `mode` : modalité qui apparait le plus souvent
- `effectif du mode` : nombre de fois où le mode apparait
- `fréquence du mode` : fréquence d'apparition du mode
- `valeurs manquantes` : nombre de valeurs manquantes

In [ ]:
df_stats = df_clean[df_clean.columns.difference(["code"])].describe(include=['object']).T

df_stats.columns = [
    "nombre de valeurs",
    "nombre de modalités",
    "mode",
    "effectif du mode",
]

df_stats["fréquence du mode"] = df_stats["effectif du mode"] / df_stats["nombre de valeurs"]

# On ajoute le nombre de valeurs manquantes
inconnu_nb = []
for c in df_stats.index:
    inconnu_nb.append(df_clean[c].isna().sum())

df_stats["valeurs manquantes"] = inconnu_nb

display(df_stats)

Excepté `nutriscore_grade`, `pnns_groups_1` et `pnns_groups_2`, les autres variables ont toutes un grand nombre de modalités. Ceci est du au fait que ce sont des variables de type texte. Elles ne respectent pas de format spécifique et sont donc très irrégulières. Par exemple, pour représenter la valeur `Royaume-Uni`, on pourrait avoir dans nos données :
- Royaume-Uni
- royaume uni,france
- UK
- etc...

On rappelle aussi que certaines variables, comme `ingredients_text`, sont au format csv et représentent des listes de textes. Cela explique aussi pourquoi il y a de si grands nombres de modalités.

### Distribution de la variable `creator`

In [ ]:
categ_plot_pie_chart(df_clean, "creator", 4)

In [ ]:
categ_plot_bar_chart(df_clean, "creator", 8)

57% des produits proviendraient du département de l'agriculture des US.

On a ensuite 16% des produits qui ont été rentré par `kiliweb` sur qui nous n'avont pas beaucoup d'information.

On retrouve ensuite des contributeurs anonymes avec 8% des produits.

### Distribution de la variable `product_name`

In [ ]:
get_cat_var_emp_dist_df(df_clean, "product_name", k=15)

Les noms qui apparaissent le plus souvent sont `Ice cream` et `Potato chips`. Les noms sont en anglais, il s'agit donc probablement de produits vendus aux US. Il est cependant vrai que, même en France, le rayons des chips comporte de nombreuses variétés et marques différentes.

La fréquence des effectifs reste très faible. En effet, le nom de chaque produit peut varier en fonction des ses particularités, de son pays de vente etc... Il faudrait donc nettoyer ces valeurs et les uniformiser afin de pouvoir les exploiter.

### Distribution de la variable `countries_fr`

In [ ]:
categ_plot_pie_chart(df_clean, "countries_fr", 2)

In [ ]:
categ_plot_bar_chart(df_clean, "countries_fr", 8)

59% des produits de notre jeu de données nettoyé sont vendus aux US et 23% en France.

Dans le jeu de données inital (avant nettoyage) ces chiffres étaient presques inversés. Il y a donc beaucoup de produits vendus en France qui ont des valeurs manquantes/aberrantes et qui ont été supprimés durant le nettoyage des données.

### Distribution de la variable `brands`

In [ ]:
tmp = df_clean[["brands"]].fillna("Unknown")

get_cat_var_emp_dist_df(tmp, "brands", k=10)

Les 5 marques les plus représentées sont française. Cependant, la variable `brands` contient 28% de valeurs manquante. Voyons dans quel pays sont vendus les produits n'ayant pas leur marque :

In [ ]:
pd.DataFrame(df_clean[df_clean["brands"].isna()]["countries_fr"].value_counts()[:5]).rename(
    columns={"countries_fr": "Nombre de produits sans marque"}
)

On constate que la grande majorité des produits n'ayant pas de valeur dans la variable `brands` sont vendus aux US.

### Distribution de la variable `pnns_groups_1`

In [ ]:
tmp = df_clean[["pnns_groups_1"]].fillna("Unknown")

categ_plot_pie_chart(tmp, "pnns_groups_1", 10)

In [ ]:
categ_plot_bar_chart(tmp, "pnns_groups_1", 10)

17% des produits sont des snacks sucrés.

Il est à noter que la catégorie la moins représentée est composée de 11000 produits. Il s'agit d'un nombre important de produits et on va donc considérer que chaque catégorie est représentative de sa population.

### Distribution de la variable `pnns_groups_2`

In [ ]:
tmp = df_clean[["pnns_groups_2"]].fillna("Unknown")

categ_plot_pie_chart(tmp, "pnns_groups_2", 15)

In [ ]:
categ_plot_bar_chart(tmp, "pnns_groups_2", 40, log=True)

Comme précédemment, on retrouve en tête du classement les biscuits et les gateaux avec 9% des produits et les sucreries avec 7% des produits.

On remaque cependant que le nombre de produits par catégorie décroit de façon exponentielle. Les dernières catégories contiennent moins de 1000 individus contre 33000 pour la première catégorie.

Nous allons privilégier `pnns_groups_1` pour le reste de cette analyse car elle a une répartition plus uniforme des individus par catégorie de produit.

### Distribution de la variable `nutriscore_grade`

In [ ]:
categ_plot_pie_chart(df_clean, "nutriscore_grade", 5)

In [ ]:
categ_plot_bar_chart(df_clean, "nutriscore_grade", 5)

Cette variable représente le Nutri-Score (code couleur avec les lettre A, B, C, D ou E) visible sur la plupart des embalages des produits. Le grade **d** semble le plus représenté avec 29% des produits.

Le grade le moins représenté est le grade **b** mais il est quand même attribué à environ 54000 produits.

### Distribution de la variable `ingredients_text`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des ingrédients.
tmp = pd.read_csv("df_clean_ingredients.csv", low_memory=False)
tmp.head(2)

get_cat_var_emp_dist_df(tmp, "ingredient", k=8)

Les ingrédients les plus utilisés sont le sel, le sucre et l'eau. On remarque que les 6 modalités les plus représentés sont des doublons : les 3 premières sont en anglais et les 3 suivantes sont en français. Ceci illustre bien le fait qu'il y a un gros travail à faire sur les valeurs des ingrédients afin de pouvoir exploiter cette variable. Nous ne l'utiliseront pas pour la suite de cette analyse.

### Distribution de la variable `additives_fr`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des additifs.
tmp = pd.read_csv("df_clean_additifs.csv", low_memory=False)
tmp.head(2)

categ_plot_pie_chart(tmp, "additif", 10)

In [ ]:
categ_plot_bar_chart(tmp, "additif", 20, log=True)

L'additif le plus utilisé est l'acide citrique qui est présent dans 8% des produits ayant des additifs. Ceci n'est pas étonnant puisqu'il s'agit d'un assaisonnement très fréquemment utilisé en cuisine (par exemple via l'utilisation du ju de citron).

In [ ]:
pct = df_clean[df_clean["additives_n"] > 0].shape[0] * 100 / df_clean.shape[0]

print_md(f"""Il est a noté que les chiffres ci-dessus concernent les produits ayant des additifs, 
soit {int(pct)}% des produits du jeu de données.""")

### Distribution de la variable `allergens`

In [ ]:
# On charge le DataFrame créé lors du nettoyage des données et
# où on avait explosé la liste des allergènes.
tmp = pd.read_csv("df_clean_allergens.csv", low_memory=False)
tmp.head(2)

get_cat_var_emp_dist_df(tmp, "allergen", k=6)

29% des allergène sont de type produit laitier. En retrouve ensuite le gluten avec 24% et le soja avec 10% des allergènes.

In [ ]:
pct = df_clean[df_clean["allergens_n"] > 0].shape[0] * 100 / df_clean.shape[0]

print_md(f"""Il est a noté que les chiffres ci-dessus concernent les produits ayant des allergènes, 
soit {int(pct)}% des produits du jeu de données.""")

### Comparaison entre la France et les US

Les 2 pays les plus représentés sont la France et les US. Continuons notre analyse en observant les différences des distributions entre ces 2 pays.

### Evolution du nombre de produits ajoutés dans le jeu de données

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))

for p in ["États-Unis", "France"]:
    # On récupère les produits vendus dans le pays p
    tmp = df_clean[df_clean["countries_fr"] == p][["last_modified_t"]]. \
        sort_values("last_modified_t").copy()
    
    # On ajoute le nombre de produits présents à chaque date
    tmp[p] = 1
    tmp[p] = tmp[p].cumsum()
    
    tmp = tmp.set_index("last_modified_t")
    
    tmp.plot(ax=ax)

ax.set_title("Evolutions du nombre de produits ajoutés dans le jeu de données par pays de vente")
ax.set_xlabel("Date")
ax.set_ylabel("Nombre total de produits")
ax.set_yscale("log")
    
# plt.savefig('Evolutions du nombre de produits ajoutés dans le jeu de données par pays de vente.png', bbox_inches='tight')
plt.show()

Les premiers produits français ont été ajouté en 2012. Depuis, il y a eu des ajout de produits réguliers qui suivent une courbre exponentielle (l'échelle des ordonnées est en log).

Les premiers produits des US ont été ajoutés en 2015. On observe des pics d'ajouts de produits. Il s'agit certainement d'upload de données venant d'une autre base de données.

Les produits et leurs compositions changent régulièrement. Pour cette partie de l'analyse, nous allons donc comparer les produits ajoutés à partir de 2015. Nous allons aussi prendre au hazard 2000 produits dans chaque catégorie de produits de la variable `pnns_groups_1`.

In [ ]:
# On filtre par date de dernière modification du produit et par pays
df_sample = df_clean[df_clean["last_modified_t"].dt.year >= 2015]
df_sample = df_sample[df_sample["countries_fr"].isin(["France", "États-Unis"])]

# On va prendre au hazard 2000 produits dans chaque catégorie de produits
df_sample = df_sample.groupby(
    ["countries_fr", "pnns_groups_1"],
    group_keys=False
).apply(lambda x: x.sample(min(len(x), 2000)))

outs = []

# On affiche les distribution empirique des catégories de produits
for p in ["France", "États-Unis"]:
    outs.append(widgets.Output())

    with outs[-1]:
        tmp = get_cat_var_emp_dist_df(
            df_sample[df_sample["countries_fr"] == p],
            "pnns_groups_1"
        )

        print_md(f"<center>Distribution empirique des </br>modalités de pnns_groups_1 dans le pays {p}</center>")
        display(tmp)

hbox = widgets.HBox(outs)
hbox.layout.justify_content="space-around"

display(hbox)

### Répartition des nutriments

In [ ]:
def num_box_chart(
        df,
        num_cols,
        cat_col,
        cat_include,
        title="",
        xlabel="",
        ylabel="",
        save=False
    ):
    # On filtre les données numériques et la catégorie qui nous intéresse
    tmp = df[df[cat_col].isin(cat_include)][num_cols + [cat_col]]

    # On transforme les colonnes numériques en catégorie
    tmp = tmp.melt(
        id_vars=cat_col,
        var_name=xlabel,
        value_name=ylabel
    )

    fig, ax = plt.subplots(figsize=(16, 10))
    
    meanprops = {
        'marker':'o',
        'markeredgecolor':'black',
        'markerfacecolor':'firebrick'
    }

    sns.boxplot(
        data=tmp,
        x=xlabel,
        y=ylabel,
        hue=cat_col,
        showfliers=False,
        showmeans=True,
        meanprops=meanprops,
        ax=ax
    )
    
    ax.set_title(title)
    ax.set_xlabel("")

    if save:
        plt.savefig(f"{title}.png", bbox_inches='tight')
    
    plt.show()

num_cols = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g"
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition des nutriments par pays",
    xlabel="type de nutriment",
    ylabel="nutriments en g",
#     save=True
)

On constate que les répartition des proportions entre les différents types de nutriments sont similaires entre les 2 pays.

On notera qu'il y a une plus grande proportion de produits aux US qui contiennent plus de glucides qu'en France.

### Répartition du nombre d'ingrédients

In [ ]:
num_cols = [
    "ingredients_n",
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition du nombre d'ingrédients par pays",
    xlabel="ingrédients",
    ylabel="nombre d'ingrédients"
)

On constate qu'il y a un peu plus d'ingrédients dans les produits vendus aux US qu'en France :
- Aux **US**, 75% des produits ont moins de **19 ingrédients**.
- En **France** 75% des produits ont moins de **14 ingrédients**.

### Répartition du nombre d'additifs et d'allergènes

In [ ]:
num_cols = [
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
]

cat_col = "countries_fr"
cat_include = ["États-Unis", "France"]

num_box_chart(
    df_sample,
    num_cols=num_cols,
    cat_col=cat_col,
    cat_include=cat_include,
    title="Répartition du nombre d'additifs et d'allergènes par pays",
    xlabel="additifs et allergènes",
    ylabel="nombre d'additifs ou d'allergènes",
#     save=True
)

25% des produits vendus en France ont au moins 1 allergène.

# Analyse multivariée

## Section

### Correlation entre `nutriscore_score` et `nutriscore_grade`

In [ ]:
num_categ_box_chart(
    df_clean,
    "nutriscore_score",
    "nutriscore_grade",
    title="Répartition des valeurs de nutriscore_score en fonction de nutriscore_grade",
    xlabel="Nutri-Score",
    ylabel="Score"
)

Il semble effectivement qu'il y ait une relation linéaire entre `nutriscore_score` et `nutriscore_grade`.

Nous allons effectuer une analyse de la variance (ANOVA) afin de quantifier cette corrélation.

In [ ]:
# Rapport de corrélation
def eta_squared(df, x, y):
    tmp = df.dropna(subset=[x, y])
    x, y = tmp[x], tmp[y]
    
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    # Variation totale
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    
    # Variation interclasse
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    
    return SCE/SCT
    
c = eta_squared(df_clean, "nutriscore_grade", "nutriscore_score")
print_md(f"Le rapport de correlation entre `nutriscore_score` et `nutriscore_grade` est de : **{c:0.2f}**")

Il y a clairement une forte correlation entre ces 2 variables.

On remarque cepdendant la présence d'outliers, notamment pour les nutriscore de grade **a** et **b**. Voyons quelques exemples de ces produits :

In [ ]:
cols = [
    "product_name",
    "countries_fr",
    "pnns_groups_1",
    "nutriscore_grade",
    "nutriscore_score",
    "ingredients_text"
]

df_clean[(df_clean["nutriscore_grade"] == "a") & (df_clean["nutriscore_score"] > 4)][cols]

On remarque que la plupart des outliers du grade **a** sont des eaux minérales ([en savoir plus sur le calcul du Nutri-Score](https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score)).

### Analyse des correlations

### Correlations entre les variables de type numérique et les variables de type catégorie

In [ ]:
rows = [
    "nutriscore_grade",
    "pnns_groups_1",
    "pnns_groups_2",
]

cols = [
    "nutriscore_score",
    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
]

data = {}

for r in rows:
    data[r] = []
    for c in cols:
        es = eta_squared(df_clean, r, c)
        data[r].append(es)

tmp = pd.DataFrame.from_dict(data, orient='index', columns=cols)

fig, ax = plt.subplots(figsize=(6, 6))

sns.heatmap(tmp.T, annot=True, ax=ax)

ax.set_title("Matrice des corrélations entre les variables de type numérique et 3 variables de type catégorie")

plt.show()

On observe des correlations modérées entre `nutriscore_score`, `energy_100g`, `carbohydrates_100g` et les catégorie de produits. On remarque que plus on a de modalités dans les catégories (`pnns_groups_2`), plus ces correlations sont fortes.

Cela semble cohérent car certaines catégories de produits, comme les snacks sucrés, vont avoir une énergie aux 100g très élevée et un nutriscore très mauvais. Alors que des catégories comme les fruits et les légumes vont certainement regrouper des produits ayant une énergie aux 100g faible et un très bon nutriscore.

In [ ]:
num_categ_box_chart(
    df_clean,
    "energy_100g",
    "pnns_groups_1",
    title="Répartition des valeurs de energy_100g en fonction de pnns_groups_1",
    xlabel="Catégorie des produits",
    ylabel="Energie en kJ",
    showfliers=False,
    rotation=30
)

On constate effectivement que la majorité des fruits et légumes ont une énergie aux 100g assez basse contrairement aux snacks sucrés.

In [ ]:
num_categ_box_chart(
    df_clean,
    "nutriscore_score",
    "pnns_groups_1",
    title="Répartition des valeurs de nutriscore_score en fonction de pnns_groups_1",
    xlabel="Catégorie des produits",
    ylabel="Score de nutrition",
    showfliers=False,
    rotation=30
)

On constate de même que le nutriscore de la majorité des fruits et légumes est bas contrairement à celui des snacks sucrés.

### Correlations entre les variables de type numérique

In [ ]:
cols = [
    "nutriscore_score",
    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
]

corr = df_clean[cols].corr()

fig, ax = plt.subplots(figsize=(14, 12))

sns.heatmap(corr, annot=True, ax=ax)
ax.set_title("Matrice des correlations des variables de type numérique")

plt.show()

Les coefficients de correlation ci-dessus nous donne les informations suivantes :
- Correlation positive élevée entre le nombre d'ingrédients et le nombre d'additifs
- Correlation positive élevée entre le nombre d'additifs et le nombre d'additifs ayant un risque élevé pour la santé
- Corrélations positives modérées entre le score de nutrition et :
    - L'énergie aux 100g
    - Les matières grasses
    - Les matières grasses saturées
    - Les sucres

### Correlations entre les variables de type catégorie

In [ ]:
def corr_cat_cat_heatmap(df, c1, c2, title):
    # On supprimes les individus ayant des valeurs manquantes
    tmp = df.dropna(subset=[c1, c2])

    # On crée le tableau de contingence
    cont = tmp[[c1, c2]].pivot_table(
        index=c1,
        columns=c2,
        aggfunc=len,
        margins=True,
        margins_name="Total",
    )
    cont

    tx = cont.loc[:,["Total"]]
    ty = cont.loc[["Total"],:]
    n = len(tmp)
    indep = tx.dot(ty) / n

    c = cont.fillna(0) # On remplace les valeurs nulles par 0
    measure = (c - indep) ** 2 / indep
    xi_n = measure.sum().sum()
    table = measure / xi_n
    
    h = 0.6 * len(cont)
    fig, ax = plt.subplots(figsize=(14, h))
    
    sns.heatmap(table.iloc[:-1,:-1], annot=c.iloc[:-1,:-1], ax=ax)
#     sns.heatmap(table.iloc[:-1,:-1], annot=table.iloc[:-1,:-1], ax=ax)
    
    ax.set_title(title)
    
    plt.show()

corr_cat_cat_heatmap(
    df_clean,
    "pnns_groups_1",
    "nutriscore_grade",
    "Matrice des correlations entre les modalités des variables pnns_groups_1 et nutriscore_grade"
)

Comme nous l'avons supposé, on observe effectivement des correlations entre les catégories des produits et le Nutri-Score. On remarquera notamment :
- Corrélation entre la catégorie **céréales et pommes de terre** et le Nutri-Score **a**.
- Corrélation entre la catégorie **Fruits et légumes** et le Nutri-Score **a**.
- Corrélation entre la catégorie **Snacks sucrés** et le Nutri-Score **e**.

Il est aussi intéressant de constater qu'il y a des produits du type **Fruits et légumes** qui ont un grade **d** ! Observons quelques-un des ces produits :

In [ ]:
cols = [
    "product_name",
    "countries_fr",
    "pnns_groups_1",
    "nutriscore_grade",
    "ingredients_text",
    "energy_100g",
]

df_clean[(df_clean["pnns_groups_1"] == "Fruits and vegetables") & (df_clean["nutriscore_grade"] == "d")][cols]

Il sagit de produits riches (comme les oléagineux) mais constitués principalement de fruits et de légumes.

### Analyse en composantes principales (ACP) 

In [ ]:
cols = [
    "nutriscore_score",

    "ingredients_n",
    "additives_n",
    "additifs_n_risque_faible",
    "additifs_n_risque_moyen",
    "additifs_n_risque_eleve",
    "allergens_n",

    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    
    "pn_beverages",
    "pn_cereals_and_potatoes",
    "pn_composite_foods",
    "pn_fat_and_sauces",
    "pn_fish_meat_eggs",
    "pn_fruits_and_vegetables",
    "pn_milk_and_dairy_products",
    "pn_salty_snacks",
    "pn_sugary_snacks",
]

# On ne conserve que les individus ayant toutes les valeurs des variables ci-dessus
acp_df = df_clean.dropna(subset=cols)

In [ ]:
X = acp_df[cols].values
names = acp_df["code"].to_numpy()
features = cols

# On transforme nos variables en variables centrées réduites
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

In [ ]:
# On réalise une PCA
pca = decomposition.PCA(n_components=len(features), random_state=0)
pca.fit(X_scaled)

# On récupère le ratio de l'inertie de chaque axe d'inertie
evrs = pca.explained_variance_ratio_

# On crée et on affiche le diagramme des éboulis de valeurs propres
fig, ax = plt.subplots(figsize=(16,10))

ax.set_title("Diagramme des éboulis de valeurs propres avec somme cumulée")
ax.set_xlabel("Rang de l'axe d'inertie")
ax.set_ylabel("Taux d'inertie")

ax.bar(np.arange(len(evrs)) + 1, evrs)
l1 = ax.plot(np.arange(len(evrs)) + 1, evrs.cumsum(), c="red", marker='o', label="Inertie cumulée")[0]

# Ligne représentant le critère de Kaiser
k = 1 / len(features)
l2 = ax.axhline(k, color="grey", ls="--", alpha=0.7, label="Critère de Kaiser")

# On ajoute le taux d'inertie
for i, evr in enumerate(evrs):
    if evr > k:
        ax.text(i + 0.65, evr + .02, f"{evr:0.2f}")
        
plt.legend(handles=[l1, l2], loc='upper left')

plt.show()

Les 4 premiers axes d'inertie permettent d'expliquer 42% de la variance totale. Les autres axes ont un taux d'intertie trop faible et qui passe rapidement en dessous du critère de Kaiser.

Nous allons continuer cette ACP avec les 4 premiers axes d'inertie F1, F2, F3 et F4. Commençons par observer nos variables projectées sur le 1er plan factoriel (F2, F1).

In [ ]:
def pca_plot_var(pcs, x_idx, y_idx, circle=False):
    # On affiche le cercle de corrélation de chaque plan factoriel
    fig, ax = plt.subplots(figsize=(14, 14))

    for j, (x, y) in enumerate(zip(pcs[x_idx, :], pcs[y_idx, :])):
        ax.plot([0, x], [0, y], color="grey", alpha=0.7)
        plt.text(x, y, cols[j], fontsize=14)

    if circle:
        ax.add_patch(plt.Circle((0, 0), 1, color="grey", alpha=0.7, fill=False))
    
    ax.axhline(0, color="grey", ls="--", alpha=0.5)
    ax.axvline(0, color="grey", ls="--", alpha=0.5)

    ax.set_title(f"Cercle des corrélations sur le plan factoriel (F{x_idx + 1}, F{y_idx + 1})")
    ax.set_xlabel(f"F{x_idx + 1} ({evrs[x_idx] * 100: 0.1f}%)")
    ax.set_ylabel(f"F{y_idx + 1} ({evrs[y_idx] * 100: 0.1f}%)")
    
    ax.set_aspect("equal")

    plt.show()

pcs = pca.components_
X_projected = pca.transform(X_scaled)
    
pca_plot_var(pcs, 1, 0)

On constate que les variables qui contribuent le plus à F1 sont `energy_100g`, `nutriscore_score`. On retrouve ensuite `pn_sugary_snacks`, `saturated-fat_100g` et `fat_100g`. On ramqaure que `pn_fruits_and_vegetables` apporte une faible contribution négative. F1 pourrait donc représenter la notion de richesse énergétique d'un produit en termes de matières grasses et de sucres.

F2 augmente avec `additives_n` et diminue avec `fat_100g`, `saturated-fat_100g` et `proteins_100g`. F2 pourrait ainsi représenter les additifs qui feraient diminuer la quantité de matières grasses. F2 pourrait aussi simplement représenter les produits qui se différencie ou non des huiles qui sont des produits riches et ayant peu d'ingrédients.

Voyons maintenant la projection des variables initiales sur (F4, F3)

In [ ]:
pca_plot_var(pcs, 3, 2)

F3 augmente avec `ingredients_n` et `pn_composite_foods` et diminue avec `sugars_100g`. F3 semble représenter les produits salés ayant de nombreux ingrédients.

F4 augmente un peu avec `additifs_n_risque_faible` et `additifs_n_risque_moyen` et diminue avec `pn_cereals_and_potatoes` et `carbohydrates_100g`. F4 pourrait indiquer la proportion d'additifs qui permettrait de remplacer les glucides.

In [ ]:
tmp = acp_df[cols].copy()
for i in range(4):
    tmp[f"F{i + 1}"] = X_projected[:, i]

corrMatrix = tmp.corr()

fig, ax = plt.subplots(figsize=(14, 14))

sns.heatmap(corrMatrix.iloc[:-4, -4:], annot=True, ax=ax)
ax.set_title("Matrice des corrélation avec les axes d'inertie de l'ACP")

plt.show()

On peut constater sur la matrice des corrélation ci-dessus, ce que l'on avait observé sur les cercles des corrélations.

Nos nouvelles variables F1, F2, F3 et F4 sont des combinaisons linéaires des variables initiales. Les interprétations ci-dessus ne sont que des hypothèses faites à partir de l'observation des poids des variables initiales dans le calcul des variables F1, F2, F3 et F4. On pourrait vérifier ces hypothèses en projetant des individus dans nos plans factoriels. Voyons donc quelques produits projetés sur les 3 premiers axes d'inerties F1, F2 et F3 :

In [ ]:
X_projected = pca.transform(X_scaled)

# On crée un DataFrame avec nos individus et en leurs ajoutant les variables F1, F2 et F3
tmp = pd.DataFrame(X_projected[:, :3], columns=[f"F{i + 1}" for i in range(3)])
tmp["nutriscore_grade"] = acp_df["nutriscore_grade"].values
tmp["pnns_groups_1"] = acp_df["pnns_groups_1"].values
tmp = tmp.dropna()

# On va prendre au hazard 100 produits pour chaque Nutri-Score.
# Il serait trop lourd d'afficher tous les individus sur un digramme interractif en 3D.
tmp = tmp.groupby(
    ["nutriscore_grade", "pnns_groups_1"],
    group_keys=False
).apply(lambda x: x.sample(min(len(x), 100)))

# On affiche les individus en 3D
fig = px.scatter_3d(
    tmp,
    x="F1",
    y="F2",
    z="F3",
    color="pnns_groups_1",
    title=f"Représentation d'un échantillon des individus projetés sur (F1, F2, F3)"
)
fig = go.FigureWidget(fig)
# display(fig)
HTML(fig.to_html())

En faisant tourner la figure ci-dessus, on remarque que certains groupes de produits sont facilement discernables comme par exemple :
- Les poissons, viandes et oeufs.
- Les boissons.
- Les produits composés.
- Les snacks sucrés.

Si on prend par exemple les individus extrêmes sur F1, on obtient :
- Un fruit ou légume avec une valeur négative sur F1.
- Un snack sucré avec une valeur positive sur F1.

Cela semble cohérent au vu de l'interprétation de F1 qui indiquerait la richesse énergétique d'un produit en termes de sucres et de matières grasses.

Dans le cadre de ce projet, les nouvelles variables crées grâce aux axes d'intertie pourraient servir de filtre lorsque les agents de Santé publique France voudront explorer le jeu de données.